# 04. Clasificación con MLP
Soluciones a los ejercicios de clasificación.

Este notebook implementa cuatro ejemplos de clasificación usando PyTorch.

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from torcheval.metrics import BinaryAccuracy, BinaryConfusionMatrix, MulticlassConfusionMatrix
from sklearn.datasets import make_blobs, make_circles, make_moons
import pandas as pd
import matplotlib.pyplot as plt


## Funciones auxiliares

In [ ]:
def train_model(model, dataloader, loss_fn, optimizer, epochs=10):
    for _ in range(epochs):
        model.train()
        for features, targets in dataloader:
            out = model(features)
            loss = loss_fn(out, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


## Ejercicio 1 - `make_blobs` con cuatro categorías

In [ ]:
X, y = make_blobs(centers=4, cluster_std=0.8)

x_mean, x_std = X.mean(), X.std()
X_norm = (X - x_mean) / x_std

tensor_X = torch.tensor(X_norm, dtype=torch.float32)
tensor_y = torch.tensor(y, dtype=torch.long)
dataset = TensorDataset(tensor_X, tensor_y)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

class MLP(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(num_features, 50),
            torch.nn.ReLU(),
            torch.nn.Linear(50, 25),
            torch.nn.ReLU(),
            torch.nn.Linear(25, num_classes)
        )
    def forward(self, x):
        return self.layers(x)

model = MLP(num_features=2, num_classes=4)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
train_model(model, dataloader, criterion, optimizer, epochs=20)

model.eval()
preds = torch.argmax(model(tensor_X), dim=1)
cm = MulticlassConfusionMatrix(num_classes=4)
cm.update(preds, tensor_y)
print(cm.compute())

## Ejercicio 2 - `make_circles`

In [ ]:
import numpy as np
# generate two circle datasets and shift one to create four classes
X1, y1 = make_circles(noise=0.1, factor=0.2)
X2, y2 = make_circles(noise=0.1, factor=0.2)
X2 += np.array([3.0, 3.0])
# adjust labels so classes become 0,1,2,3
y2 += 2
X = np.vstack([X1, X2])
y = np.concatenate([y1, y2])

X_norm = (X - X.mean(axis=0)) / X.std(axis=0)
tensor_X = torch.tensor(X_norm, dtype=torch.float32)
tensor_y = torch.tensor(y, dtype=torch.long)

dataset = TensorDataset(tensor_X, tensor_y)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

model = MLP(num_features=2, num_classes=4)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
train_model(model, dataloader, criterion, optimizer, epochs=20)

model.eval()
preds = torch.argmax(model(tensor_X), dim=1)
cm = MulticlassConfusionMatrix(num_classes=4)
cm.update(preds, tensor_y)
print(cm.compute())



## Ejercicio 3 - `make_moons`

In [ ]:
X, y = make_moons(noise=0.1)
X_norm = (X - X.mean()) / X.std()
tensor_X = torch.tensor(X_norm, dtype=torch.float32)
tensor_y = torch.tensor(y, dtype=torch.float32)
dataset = TensorDataset(tensor_X, tensor_y)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

model = BinaryMLP(num_features=2)
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
train_model(model, dataloader, criterion, optimizer, epochs=20)

model.eval()
out = model(tensor_X)
preds = (out > 0).int()
cm = BinaryConfusionMatrix()
cm.update(preds, tensor_y.int())
print(cm.compute())

## Ejercicio 4 - Datos de infidelidad

In [ ]:
url = 'https://raw.githubusercontent.com/mcstllns/UNIR2024/main/data-affairs.csv'
data = pd.read_csv(url)
X = data.drop('affairs', axis=1)
y = data['affairs'].astype(float)

X_norm = (X - X.mean()) / X.std()
tensor_X = torch.tensor(X_norm.values, dtype=torch.float32)
tensor_y = torch.tensor(y.values, dtype=torch.float32)
dataset = TensorDataset(tensor_X, tensor_y)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

model = BinaryMLP(num_features=X.shape[1])
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
train_model(model, dataloader, criterion, optimizer, epochs=20)

model.eval()
out = model(tensor_X)
preds = (out > 0).int()
acc = BinaryAccuracy()
acc.update(preds, tensor_y.int())
print('accuracy:', acc.compute().item())